In [1]:
import pandas as pd
import pysal as ps
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import geocoder
import geopandas as gpd

In [20]:
#import median and mean prices 1992-2022 for each council in Victoria state
df = pd.read_excel(r'C:\WORK_IAAC\submissions\UrbanSense\UrbanSens_QGIS\Real Estate\Year_SummaryV2021.xls')
df.head()

,Melbourne Metropolitan Area,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Residential price statistics 1992 to 2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,House Median,NaN,NaN,NaN,NaN,NaN,Unit/Apartment Median,NaN,NaN,NaN,NaN,Vacant House Block Median,NaN,NaN,NaN,NaN
2,NaN,NaN,Houses,NaN,NaN,NaN,Units/Apartments,NaN,NaN,NaN,NaN,Vacant House Blocks,NaN,NaN,NaN,NaN
3,Year,No. of Sales,Median Price,NaN,Mean Price,NaN,No. of Sales,Median Price,NaN,Mean Price,NaN,No. of Sales,Median Price,NaN,Mean Price,NaN
4,1992,41212,125000,NaN,150106,NaN,15130,110000,NaN,133910,NaN,12924,45000,NaN,52976,NaN


In [4]:
#import metropolitan councils +
gdf = gpd.read_file(r'C:\WORK_IAAC\submissions\UrbanSense\UrbanSens_QGIS\Administrative\Municipalities_MLB.shp')
gdf.head()

,LG_PLY_PID,LGA_PID,DT_CREATE,LGA_NAME,ABB_NAME,STATE,geometry
0,lgpiDZvaGB0_ay6,lga5591321694d6,2023-05-11,Banyule City,Banyule,VIC,"POLYGON ((145.02805 -37.76410, 145.02789 -37.7..."
1,lgp3b89ab2a2346,lgaac2e88625ea2,2022-08-11,Bass Coast Shire,Bass Coast,VIC,"POLYGON ((145.34607 -38.50858, 145.34598 -38.5..."
2,lgp64b91fc10b87,lgaac2e88625ea2,2022-08-11,Bass Coast Shire,Bass Coast,VIC,"POLYGON ((145.80949 -38.62749, 145.81070 -38.6..."
3,lgp85fdcce19121,lgaac2e88625ea2,2022-08-11,Bass Coast Shire,Bass Coast,VIC,"POLYGON ((145.35873 -38.51754, 145.35834 -38.5..."
4,lgpbfubUq2vn4Wi,lga53f7b61b0ed6,2023-05-11,Bayside City,Bayside,VIC,"POLYGON ((145.00755 -37.95811, 145.00725 -37.9..."


In [28]:
ci = gdf['ABB_NAME'].unique().tolist()
del ci[9]
ci

['Banyule',
 'Bass Coast',
 'Bayside',
 'Boroondara',
 'Brimbank',
 'Cardinia',
 'Casey',
 'Darebin',
 'Frankston',
 'Glen Eira',
 'Greater Dandenong',
 'Greater Geelong',
 'Hobsons Bay',
 'Hume',
 'Kingston',
 'Knox',
 'Manningham',
 'Maribyrnong',
 'Maroondah',
 'Melbourne',
 'Melton',
 'Merri-Bek',
 'Monash',
 'Moonee Valley',
 'Mornington Peninsula',
 'Nillumbik',
 'Port Phillip',
 'Queenscliffe',
 'Stonnington',
 'Whitehorse',
 'Whittlesea',
 'Wyndham',
 'Yarra',
 'Yarra Ranges']

In [26]:
#make csvs from 1 excel
for word in ci:
    # Filter the DataFrame based on the search word in the appropriate column index
    filtered_df = df[df.iloc[:, 0].str.contains(word, na=False)]
    
    # Check if any matching rows were found
    if not filtered_df.empty:
        # Find the index of the first occurrence of the search word
        index = filtered_df.index[0]
        
        # Define the bounds for the desired portion of the DataFrame
        start_row = index + 4
        end_row = start_row + 32
        
        # Get the number of columns in the DataFrame
        num_columns = df.shape[1]
        
        # Exclude specific columns from the bounds
        excluded_columns = [3, 5, 8, 10, 13]
        included_columns = [col for col in range(num_columns) if col not in excluded_columns]
        
        # Check if there are any included columns
        if included_columns:
            # Extract the desired portion of the DataFrame
            extracted_df = df.iloc[start_row:end_row, included_columns]
            
            # Get the values of the first row
            first_row_values = extracted_df.iloc[0, :]
            
            # Create a dictionary to map old column names to new column names with suffixes
            column_mapping = {}
            for i, col in enumerate(extracted_df.columns):
                if i == 0:
                    column_mapping[col] = col  # No suffix for the first column
                elif 1 <= i <= 3:
                    column_mapping[col] = col + '_houses'
                elif 4 <= i <= 6:
                    column_mapping[col] = col + '_apartments'
                elif 7 <= i <= 9:
                    column_mapping[col] = col + '_vacplots'
            
            # Rename the columns using the column mapping dictionary
            extracted_df = extracted_df.rename(columns=column_mapping)
            
            # Set the first row values as the new column names
            extracted_df.columns = first_row_values
            
            # Create the filename for the CSV based on the search word
            filename = f"C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/{word}.csv"
            
            # Save the extracted portion of the DataFrame as a CSV file
            extracted_df.to_csv(filename, index=False)

In [30]:
# refine csvs data
import os
import pandas as pd

# Define the folder path
folder_path = "C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/selection"

# Create the folder if it doesn't exist
os.makedirs(folder_path, exist_ok=True)

# Iterate over the search words
for search_word in ci:
    # Create the filename
    filename = f"C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/{search_word}.csv"

    # Check if the file exists
    if not os.path.exists(filename):
        print(f"File '{filename}' does not exist. Skipping.")
        continue

    # Move the CSV to the 'selection' folder
    new_filepath = os.path.join(folder_path, f"{search_word}.csv")
    os.rename(filename, new_filepath)

    # Read the CSV into a DataFrame
    df1 = pd.read_csv(new_filepath)

    # Modify column names
    df1.columns = df1.columns.str.replace('.', '').str.replace(' ', '_')

    # Remove last column
    df1 = df1.iloc[:, :-1]

    # Define prefixes and ranges
    prefixes = ['homes_', 'flats_', 'plots_']
    ranges = [(2, 4), (5, 7), (8, 10)]

    # Add prefixes to column names
    new_columns = []
    for i, col_name in enumerate(df1.columns):
        for prefix, (start, end) in zip(prefixes, ranges):
            if start <= i + 1 <= end:
                col_name = prefix + col_name
                break
        new_columns.append(col_name)

    # Assign new column names to the DataFrame
    df1.columns = new_columns

    # Remove the first row
    df1 = df1.drop(0)

    # Convert values to integers
    df1 = df1.astype(int)
    
    df1.head()

File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Banyule.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Bass Coast.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Bayside.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Boroondara.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Brimbank.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Cardinia.csv' does not exist. Skipping.
File 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/Merri-Bek.csv' does not exist. Skipping.


In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31 entries, 1 to 31
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   Year                 31 non-null     int32
 1   homes_No_of_Sales    31 non-null     int32
 2   homes_Median_Price   31 non-null     int32
 3   homes_Mean_Price     31 non-null     int32
 4   flats_No_of_Sales1   31 non-null     int32
 5   flats_Median_Price1  31 non-null     int32
 6   flats_Mean_Price1    31 non-null     int32
 7   plots_No_of_Sales2   31 non-null     int32
 8   plots_Median_Price2  31 non-null     int32
 9   plots_Mean_Price2    31 non-null     int32
dtypes: int32(10)
memory usage: 1.3 KB


In [32]:
directory = 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/selection/'

# Get the names of files in the directory
file_names = os.listdir(directory)

# Get the total number of files in the directory
file_count = len(file_names)

# Print the names and amount of files
print("Files in the directory:")
for file_name in file_names:
    print(file_name)
    
print("\nTotal number of files:", file_count)

Files in the directory:
Banyule.csv
Bass Coast.csv
Bayside.csv
Boroondara.csv
Brimbank.csv
Casey.csv
Darebin.csv
Frankston.csv
Glen Eira.csv
Greater Dandenong.csv
Greater Geelong.csv
Hobsons Bay.csv
Hume.csv
Kingston.csv
Knox.csv
Manningham.csv
Maribyrnong.csv
Maroondah.csv
Melbourne.csv
Melton.csv
Monash.csv
Moonee Valley.csv
Mornington Peninsula.csv
Nillumbik.csv
Port Phillip.csv
Queenscliffe.csv
Stonnington.csv
Whitehorse.csv
Whittlesea.csv
Wyndham.csv
Yarra Ranges.csv
Yarra.csv

Total number of files: 32


In [11]:
import os
import pandas as pd

# Directory path
directory = 'C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/selection/'

# Initialize an empty list to store individual DataFrames
dfs = []

# Iterate over each file in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        # Read the CSV file
        file_path = os.path.join(directory, filename)
        df = pd.read_csv(file_path)
        
        # Extract the council name from the file name
        council = filename.split('.')[0]
        
        # Add the 'council' column with the council name
        df.insert(0, 'council', council)
        
        # Append the DataFrame to the list
        dfs.append(df)

# Merge all DataFrames into a single DataFrame
merged_df = pd.concat(dfs, ignore_index=True)
merged_df = merged_df.iloc[:, :-1]
merged_df.head()

,council,Year,No. of Sales,Median Price,Mean Price,No. of Sales.1,Median Price.1,Mean Price.1,No. of Sales.2,Median Price.2,Mean Price.2
0,Banyule,Year,No. of Sales,Median Price,Mean Price,No. of Sales,Median Price,Mean Price,No. of Sales,Median Price,Mean Price
1,Banyule,1992,1324,135000,156065,460,115000,118192,81,76000,82074
2,Banyule,1993,1405,134500,152532,479,117000,126837,164,82500,86332
3,Banyule,1994,1519,136000,156499,534,116000,124941,249,79000,80452
4,Banyule,1995,1400,133750,158410,472,120000,125445,200,83250,84765


In [12]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1024 entries, 0 to 1023
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   council         1024 non-null   object
 1   Year            1024 non-null   object
 2   No. of Sales    1024 non-null   object
 3   Median Price    1024 non-null   object
 4   Mean Price      1024 non-null   object
 5   No. of Sales.1  1024 non-null   object
 6   Median Price.1  1024 non-null   object
 7   Mean Price.1    1024 non-null   object
 8   No. of Sales.2  1024 non-null   object
 9   Median Price.2  1024 non-null   object
 10  Mean Price.2    1024 non-null   object
dtypes: object(11)
memory usage: 88.1+ KB


In [32]:
import pandas as pd
import numpy as np

# Assuming you already have the df DataFrame
dfr = merged_df
# Convert 'Year' column to numeric values
dfr['Year'] = pd.to_numeric(dfr['Year'], errors='coerce')

# Drop rows with NaN values in 'Year' column
dfr = dfr.dropna(subset=['Year'])

# Reset the index of the DataFrame
dfr = dfr.reset_index(drop=True)
dfr['Year'] = dfr['Year'].astype(int)
dfr.columns = dfr.columns.str.replace('.', '').str.replace(' ', '_')
new_column_names = {}

for i in range(2, 5):
    new_column_names[dfr.columns[i]] = 'homes_' + dfr.columns[i]

for i in range(5, 8):
    new_column_names[dfr.columns[i]] = 'flats_' + dfr.columns[i]

for i in range(8, 11):
    new_column_names[dfr.columns[i]] = 'plots_' + dfr.columns[i]

dfr = dfr.rename(columns=new_column_names)
dfr.head()

,council,Year,homes_No_of_Sales,homes_Median_Price,homes_Mean_Price,flats_No_of_Sales1,flats_Median_Price1,flats_Mean_Price1,plots_No_of_Sales2,plots_Median_Price2,plots_Mean_Price2
0,Banyule,1992,1324,135000,156065,460,115000,118192,81,76000,82074
1,Banyule,1993,1405,134500,152532,479,117000,126837,164,82500,86332
2,Banyule,1994,1519,136000,156499,534,116000,124941,249,79000,80452
3,Banyule,1995,1400,133750,158410,472,120000,125445,200,83250,84765
4,Banyule,1996,1514,133500,157426,563,117500,127004,137,84000,96124


In [33]:
dfr.to_csv('C:/WORK_IAAC/submissions/UrbanSense/UrbanSens_QGIS/Real Estate/MelMetro_realestateprices_1992-2022.csv', index=False)